In [3]:
import pandas as pd
import numpy as np
from scipy.stats import boxcox

In [4]:
data = pd.read_csv('Documents/Datasets/titanic/train.csv')

In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df = pd.get_dummies(data, columns=['Sex','Pclass'], drop_first=True)

In [6]:
df.head(3)

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_male,Pclass_2,Pclass_3
0,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,True,False,True
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,False,False,False
2,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,False,False,True


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, roc_auc_score, classification_report
from sklearn.preprocessing import PolynomialFeatures

In [7]:
x = df[['Pclass_2', 'Pclass_3', 'Sex_male']]
y = df['Survived']

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=228)

In [9]:
model = LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

In [10]:
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy*100)

76.53631284916202


In [11]:
# пробуем добавить взаимодействие признаков
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=True)
x_train_poly = poly.fit_transform(x_train)
x_test_poly = poly.transform(x_test)

In [12]:
model_poly = LogisticRegression()
model_poly.fit(x_train_poly, y_train)

LogisticRegression()

In [13]:
y_pred_poly = model_poly.predict(x_test_poly)
accuracy = accuracy_score(y_test, y_pred_poly)
print(accuracy *100)

76.53631284916202


In [14]:
# Предсказываем выживет пассажир, или нет

In [15]:
df_test = pd.read_csv('Documents/Datasets/titanic/test.csv')

In [16]:
df_test = pd.get_dummies(df_test, columns=['Sex','Pclass'], drop_first=True)

In [17]:
x_pred = df_test[['Pclass_2', 'Pclass_3', 'Sex_male']]

In [18]:
y_pred = model.predict(x_pred)

In [19]:
df_test['Survived'] = y_pred

In [20]:
m = confusion_matrix(y_test, y_pred_poly)
print(m)

[[96 15]
 [27 41]]


In [21]:
report = pd.DataFrame(classification_report(y_test, y_pred_poly, output_dict=True)).transpose()

In [22]:
report

,precision,recall,f1-score,support
0,0.780488,0.864865,0.820513,111.000000
1,0.732143,0.602941,0.661290,68.000000
accuracy,0.765363,0.765363,0.765363,0.765363
macro avg,0.756315,0.733903,0.740902,179.000000
weighted avg,0.762122,0.765363,0.760026,179.000000


In [23]:
# Недосчитались положительных предсказаний

In [24]:
# Проверяем на мультиколлинеарность
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [25]:
x = df[['Pclass_2', 'Pclass_3', 'Sex_male']]
vif = pd.DataFrame()
vif['feature'] = x.columns
vif['score'] = [variance_inflation_factor(x.values, i) for i in range(len(x.columns))]
vif

,feature,score
0,Pclass_2,0.006718
1,Pclass_3,0.003898
2,Sex_male,0.003726


In [26]:
# мультиколлинеарность крайне незначительная

In [11]:
#проверим модель с помощью GLM
import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod import families
import patsy

In [28]:
formula = 'Survived ~ C(Sex) + C(Pclass) + C(Sex):C(Pclass)'
y, x = patsy.dmatrices(formula, data=data, return_type='dataframe')

In [29]:
model = sm.GLM(y,x, family=families.Binomial()).fit()
print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               Survived   No. Observations:                  891
Model:                            GLM   Df Residuals:                      885
Model Family:                Binomial   Df Model:                            5
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -399.05
Date:                Wed, 09 Oct 2024   Deviance:                       798.10
Time:                        21:50:22   Pearson chi2:                     891.
No. Iterations:                     6   Pseudo R-squ. (CS):             0.3534
Covariance Type:            nonrobust                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

In [13]:
formula2 = 'Survived ~ C(Sex) + C(Age) + C(Pclass) + C(Sex):C(Pclass)'
y2, x2 = patsy.dmatrices(formula2, data=data, return_type='dataframe')

In [12]:
model2 = sm.GLM(y2,x2, family=families.Binomial()).fit()
# print(model2.summary())

NameError: name 'y2' is not defined

In [32]:
# Видим значительное снижение log-likelihood

In [24]:
#заполняем пропуски медианой
df['Age'] = df.Age.median()

In [30]:
x = df[['Pclass_2', 'Pclass_3', 'Sex_male', 'Age']]
y = df['Survived']

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x2,y2, test_size=0.2, random_state=228)

In [40]:
model_age = LogisticRegression()
model_age.fit(x_train,y_train)

/Users/sergejknol/miniconda3/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [41]:
y_pred_age = model_age.predict(x_test)
report = pd.DataFrame(classification_report(y_test, y_pred_age, output_dict=True)).transpose()
report

,precision,recall,f1-score,support
0.0,0.831579,0.908046,0.868132,87.000000
1.0,0.833333,0.714286,0.769231,56.000000
accuracy,0.832168,0.832168,0.832168,0.832168
macro avg,0.832456,0.811166,0.818681,143.000000
weighted avg,0.832266,0.832168,0.829401,143.000000


In [ ]:
# Видим увеличение точности модели

In [1]:
#Пробуем дерево
from sklearn import tree

In [32]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=228)

In [33]:
clf = tree.DecisionTreeClassifier(criterion='entropy')

In [34]:
clf.fit(x_train,y_train)

DecisionTreeClassifier(criterion='entropy')

In [36]:
y_pred = clf.predict(x_test)

In [38]:
report = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose()
report

,precision,recall,f1-score,support
0,0.780488,0.864865,0.820513,111.000000
1,0.732143,0.602941,0.661290,68.000000
accuracy,0.765363,0.765363,0.765363,0.765363
macro avg,0.756315,0.733903,0.740902,179.000000
weighted avg,0.762122,0.765363,0.760026,179.000000
